In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [ ]:
df_green.show()

In [ ]:
df_green.registerTempTable('green')

In [ ]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [ ]:

df_green_revenue \
    .write.parquet('data/report/revenue/green')

In [ ]:
# This time with partition since the files are quite small and we can do fewer reshuffling during the group by stage
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [ ]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [ ]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [ ]:
df_yellow_revenue \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [ ]:
# This time with partition since the files are quite small and we can do fewer reshuffling during the group by stage
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [ ]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
# If we are not reading the materialised tables, SPARK runs the entire stages once again based on the Spark SQL
# You see this in the DAG visualisation on the browser at http://localhost:4040
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.show()

In [ ]:
# Since we have materialised the outputs from the summary table into parquest files, we will use them
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [ ]:
# Since the same columns are there in both tables, we will rename then.
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [ ]:
df_join = spark.read.parquet('data/report/revenue/total')

In [ ]:
df_join

In [ ]:
spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv') \
    .write.parquet('zones')

In [ ]:
!ls -lh

In [ ]:
df_zones = spark.read.parquet('zones/')

In [ ]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [ ]:
df_result.columns

In [ ]:
df_result.show()

In [ ]:
#df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [ ]:
df_result.show()